In [1]:
import tensorflow as tf 
import tensorflow_probability as tfp

In [2]:
n = 5
m = 3
num_chains = 7

In [3]:
kernel = tf.random.normal(shape=(num_chains, n, m))
bias = tf.random.normal(shape=(num_chains, m))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-01-02 07:17:04.508050: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-02 07:17:04.508385: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
kernel

<tf.Tensor: shape=(7, 5, 3), dtype=float32, numpy=
array([[[ 8.51410925e-01,  3.80788982e-01,  1.61915040e+00],
        [ 7.99249649e-01,  1.34933531e-01,  5.29468298e-01],
        [-8.76355648e-01, -1.23933184e+00,  7.28454828e-01],
        [-1.54124260e+00,  1.76365983e+00,  2.29737252e-01],
        [-2.19800696e-01,  1.13278544e+00, -4.69487399e-01]],

       [[-5.88677227e-01,  5.89945540e-02, -1.94407380e+00],
        [-4.23632234e-01,  7.23942757e-01, -8.78119946e-01],
        [-4.40934032e-01,  1.52204013e+00,  8.38449776e-01],
        [-5.70991516e-01,  4.22989696e-01, -2.84892112e-01],
        [-3.96363676e-01, -8.88156295e-01,  1.94174215e-01]],

       [[-4.90415026e-04,  3.01149376e-02, -1.28273571e+00],
        [-3.28183889e-01, -1.92466068e+00,  9.99726892e-01],
        [ 2.15764731e-01, -1.07253718e+00, -2.37776399e-01],
        [-9.99145746e-01,  1.27894831e+00, -3.81406635e-01],
        [ 7.38356560e-02, -1.32684255e+00,  5.59095740e-01]],

       [[-1.32847452e+00,  1

In [5]:
mcmc_samples = [kernel, bias]
def get_posterior_mcmc(mcmc_samples):
    def posterior_mcmc(kernel_size, bias_size, dtype=None):
        return tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.Empirical(mcmc_samples)
        )
    return posterior_mcmc

In [6]:
posterior = get_posterior_mcmc(mcmc_samples=mcmc_samples)

In [9]:
kernel_posterior = tfp.distributions.Empirical(samples=kernel, event_ndims=2)
bias_posterior = tfp.distributions.Empirical(samples=bias, event_ndims=1)


In [10]:
print(kernel_posterior.sample())
print(kernel)


tf.Tensor(
[[-1.4596404   0.32711762  1.4388838 ]
 [-1.2530208  -1.3843281  -0.4347004 ]
 [-0.01345622 -1.8781968   2.008579  ]
 [ 0.683504    0.03089695  1.4429535 ]
 [ 0.07077185 -0.9416656   0.6513678 ]], shape=(5, 3), dtype=float32)
tf.Tensor(
[[[ 8.51410925e-01  3.80788982e-01  1.61915040e+00]
  [ 7.99249649e-01  1.34933531e-01  5.29468298e-01]
  [-8.76355648e-01 -1.23933184e+00  7.28454828e-01]
  [-1.54124260e+00  1.76365983e+00  2.29737252e-01]
  [-2.19800696e-01  1.13278544e+00 -4.69487399e-01]]

 [[-5.88677227e-01  5.89945540e-02 -1.94407380e+00]
  [-4.23632234e-01  7.23942757e-01 -8.78119946e-01]
  [-4.40934032e-01  1.52204013e+00  8.38449776e-01]
  [-5.70991516e-01  4.22989696e-01 -2.84892112e-01]
  [-3.96363676e-01 -8.88156295e-01  1.94174215e-01]]

 [[-4.90415026e-04  3.01149376e-02 -1.28273571e+00]
  [-3.28183889e-01 -1.92466068e+00  9.99726892e-01]
  [ 2.15764731e-01 -1.07253718e+00 -2.37776399e-01]
  [-9.99145746e-01  1.27894831e+00 -3.81406635e-01]
  [ 7.38356560e-02 -

In [11]:
print(bias_posterior.sample())
bias

tf.Tensor([-0.2570973  -0.95289993  0.5167044 ], shape=(3,), dtype=float32)


<tf.Tensor: shape=(7, 3), dtype=float32, numpy=
array([[-0.32625642, -1.5554326 , -1.3853469 ],
       [ 0.00318867,  0.5953822 , -0.58557576],
       [-0.5830249 ,  2.5067716 ,  1.2896934 ],
       [-0.2570973 , -0.95289993,  0.5167044 ],
       [-1.2639438 ,  0.49971095, -1.5241348 ],
       [-0.14181434,  0.41335255, -0.9111221 ],
       [-1.2150005 ,  1.9182705 ,  0.7719303 ]], dtype=float32)>

In [70]:
weights = [
    [kernel[i], bias[i]] for i in range(kernel.shape[0])
]

In [16]:
weight_posterior = tfp.distributions.JointDistributionSequential([
    tfp.distributions.Empirical(samples=kernel, event_ndims=2),
    tfp.distributions.Empirical(samples=bias, event_ndims=1),
    ],
    batch_ndims=2
)

In [19]:
print(weight_posterior.sample(5))
print(weight_posterior.batch_shape)
print(weight_posterior.event_shape)


[<tf.Tensor: shape=(5, 5, 3), dtype=float32, numpy=
array([[[ 8.51410925e-01,  3.80788982e-01,  1.61915040e+00],
        [ 7.99249649e-01,  1.34933531e-01,  5.29468298e-01],
        [-8.76355648e-01, -1.23933184e+00,  7.28454828e-01],
        [-1.54124260e+00,  1.76365983e+00,  2.29737252e-01],
        [-2.19800696e-01,  1.13278544e+00, -4.69487399e-01]],

       [[-1.32847452e+00,  1.34684667e-01,  3.32941979e-01],
        [ 2.02927089e+00,  1.08894855e-01,  2.14401307e-03],
        [ 1.08220923e+00, -4.87906843e-01, -7.61657596e-01],
        [-6.91805482e-01,  9.12398517e-01,  1.07372987e+00],
        [ 1.48202693e+00,  1.17562795e+00,  7.47611582e-01]],

       [[ 1.76275039e+00, -1.07951176e+00, -8.60032022e-01],
        [-9.76063609e-02, -1.46054780e+00, -1.07729959e+00],
        [ 1.54099941e+00,  1.63863468e+00,  2.74887383e-01],
        [-8.08984816e-01,  5.64929187e-01, -2.49989390e+00],
        [-1.72894609e+00, -2.07101393e+00, -8.36721838e-01]],

       [[-5.88677227e-01,  

In [18]:
print(kernel)
print(bias)

tf.Tensor(
[[[ 8.51410925e-01  3.80788982e-01  1.61915040e+00]
  [ 7.99249649e-01  1.34933531e-01  5.29468298e-01]
  [-8.76355648e-01 -1.23933184e+00  7.28454828e-01]
  [-1.54124260e+00  1.76365983e+00  2.29737252e-01]
  [-2.19800696e-01  1.13278544e+00 -4.69487399e-01]]

 [[-5.88677227e-01  5.89945540e-02 -1.94407380e+00]
  [-4.23632234e-01  7.23942757e-01 -8.78119946e-01]
  [-4.40934032e-01  1.52204013e+00  8.38449776e-01]
  [-5.70991516e-01  4.22989696e-01 -2.84892112e-01]
  [-3.96363676e-01 -8.88156295e-01  1.94174215e-01]]

 [[-4.90415026e-04  3.01149376e-02 -1.28273571e+00]
  [-3.28183889e-01 -1.92466068e+00  9.99726892e-01]
  [ 2.15764731e-01 -1.07253718e+00 -2.37776399e-01]
  [-9.99145746e-01  1.27894831e+00 -3.81406635e-01]
  [ 7.38356560e-02 -1.32684255e+00  5.59095740e-01]]

 [[-1.32847452e+00  1.34684667e-01  3.32941979e-01]
  [ 2.02927089e+00  1.08894855e-01  2.14401307e-03]
  [ 1.08220923e+00 -4.87906843e-01 -7.61657596e-01]
  [-6.91805482e-01  9.12398517e-01  1.07372987e

In [21]:
print(weight_posterior.sample(7))

[<tf.Tensor: shape=(7, 5, 3), dtype=float32, numpy=
array([[[-5.88677227e-01,  5.89945540e-02, -1.94407380e+00],
        [-4.23632234e-01,  7.23942757e-01, -8.78119946e-01],
        [-4.40934032e-01,  1.52204013e+00,  8.38449776e-01],
        [-5.70991516e-01,  4.22989696e-01, -2.84892112e-01],
        [-3.96363676e-01, -8.88156295e-01,  1.94174215e-01]],

       [[-1.32847452e+00,  1.34684667e-01,  3.32941979e-01],
        [ 2.02927089e+00,  1.08894855e-01,  2.14401307e-03],
        [ 1.08220923e+00, -4.87906843e-01, -7.61657596e-01],
        [-6.91805482e-01,  9.12398517e-01,  1.07372987e+00],
        [ 1.48202693e+00,  1.17562795e+00,  7.47611582e-01]],

       [[ 3.01989138e-01,  4.99761730e-01,  1.71117795e+00],
        [-1.14835054e-01,  1.12909925e+00, -6.68139398e-01],
        [ 1.00720477e+00, -1.24690914e+00, -2.54817319e+00],
        [-8.32141161e-01,  6.53460145e-01,  3.37305576e-01],
        [ 4.73907769e-01, -6.55781209e-01,  1.47173464e+00]],

       [[-5.88677227e-01,  

In [22]:
kernel_prior = tfp.distributions.Normal(loc=0., scale=1.)
bias_prior = tfp.distributions.Normal(loc=0., scale=1.)
weight_prior = tfp.distributions.JointDistributionSequential([
    kernel_prior,
    bias_prior
])

In [23]:
tfk = tf.keras
tfkl = tf.keras.layers
tfd = tfp.distributions
tfpl = tfp.layers

In [43]:
x = tf.random.normal(shape=(1, 5))

In [44]:
model.call(x)

TypeError: Exception encountered when calling layer "distribution_lambda_4" (type DistributionLambda).

[[[ 8.51410925e-01  3.80788982e-01  1.61915040e+00]
  [ 7.99249649e-01  1.34933531e-01  5.29468298e-01]
  [-8.76355648e-01 -1.23933184e+00  7.28454828e-01]
  [-1.54124260e+00  1.76365983e+00  2.29737252e-01]
  [-2.19800696e-01  1.13278544e+00 -4.69487399e-01]]

 [[-5.88677227e-01  5.89945540e-02 -1.94407380e+00]
  [-4.23632234e-01  7.23942757e-01 -8.78119946e-01]
  [-4.40934032e-01  1.52204013e+00  8.38449776e-01]
  [-5.70991516e-01  4.22989696e-01 -2.84892112e-01]
  [-3.96363676e-01 -8.88156295e-01  1.94174215e-01]]

 [[-4.90415026e-04  3.01149376e-02 -1.28273571e+00]
  [-3.28183889e-01 -1.92466068e+00  9.99726892e-01]
  [ 2.15764731e-01 -1.07253718e+00 -2.37776399e-01]
  [-9.99145746e-01  1.27894831e+00 -3.81406635e-01]
  [ 7.38356560e-02 -1.32684255e+00  5.59095740e-01]]

 [[-1.32847452e+00  1.34684667e-01  3.32941979e-01]
  [ 2.02927089e+00  1.08894855e-01  2.14401307e-03]
  [ 1.